In [109]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [110]:
page_dataframes = []

In [111]:
current_page_url = "https://quotes.toscrape.com/page/" + str(1)

page_response = requests.get(current_page_url)

page_soup = BeautifulSoup(page_response.content, 'html.parser')

quote_blocks = page_soup.find_all('div', class_='quote')

quote_blocks

[<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
 <span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
 <span>by <small class="author" itemprop="author">Albert Einstein</small>
 <a href="/author/Albert-Einstein">(about)</a>
 </span>
 <div class="tags">
             Tags:
             <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
 <a class="tag" href="/tag/change/page/1/">change</a>
 <a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
 <a class="tag" href="/tag/thinking/page/1/">thinking</a>
 <a class="tag" href="/tag/world/page/1/">world</a>
 </div>
 </div>,
 <div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
 <span class="text" itemprop="text">“It is our choices, Harry, that show what we truly are, far more than our abilities.”</span>
 <span>by <small class="author" itempr

In [112]:
quote_text = quote_blocks[0].find("span", class_="text").text
author_name = quote_blocks[0].find('small', class_='author').text
tags = quote_blocks[0].find('div', class_='tags').meta['content']

quote_text,  author_name, tags

('“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
 'Albert Einstein',
 'change,deep-thoughts,thinking,world')

In [113]:
page_dataframes = []
for page in range(1, 11):
    current_page_url = "https://quotes.toscrape.com/page/" + str(page)

    page_response = requests.get(current_page_url)

    page_soup = BeautifulSoup(page_response.content, 'html.parser')

    quote_blocks = page_soup.find_all('div', class_='quote')

    quotes_on_page = []

    for block in quote_blocks:
        quote_text = block.find("span", class_="text").text
        author_name = block.find('small', class_='author').text
        tags = block.find('div', class_='tags').meta['content']

        author_page_url = "https://quotes.toscrape.com/" + block.find("a")["href"]

        author_response = requests.get(author_page_url)
        author_soup = BeautifulSoup(author_response.text, 'html.parser')

        birth_date = author_soup.find("span", class_="author-born-date").text
        birth_location = author_soup.find("span", class_="author-born-location").text
        bio_description = author_soup.find("div", class_="author-description").get_text().strip()

        quotes_on_page.append((quote_text, author_name, tags, birth_date, birth_location, bio_description))

    page_df = pd.DataFrame(quotes_on_page, columns=["Quote", "Author", "Tags", "Birth Date", "Birth Location", "Biography"])

    page_dataframes.append(page_df)


In [114]:
final_df = pd.concat(page_dataframes, ignore_index=True)


In [115]:
final_df

,Quote,Author,Tags,Birth Date,Birth Location,Biography
0,“The world as we have created it is a process ...,Albert Einstein,"change,deep-thoughts,thinking,world","March 14, 1879","in Ulm, Germany","In 1879, Albert Einstein was born in Ulm, Germ..."
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"abilities,choices","July 31, 1965","in Yate, South Gloucestershire, England, The U...",See also: Robert GalbraithAlthough she writes ...
2,“There are only two ways to live your life. On...,Albert Einstein,"inspirational,life,live,miracle,miracles","March 14, 1879","in Ulm, Germany","In 1879, Albert Einstein was born in Ulm, Germ..."
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"aliteracy,books,classic,humor","December 16, 1775","in Steventon Rectory, Hampshire, The United Ki...",Jane Austen was an English novelist whose work...
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"be-yourself,inspirational","June 01, 1926",in The United States,Marilyn Monroe (born Norma Jeane Mortenson; Ju...
...,...,...,...,...,...,...
95,“You never really understand a person until yo...,Harper Lee,better-life-empathy,"April 28, 1926","in Monroeville, Alabama, The United States","Harper Lee, known as Nelle, was born in the Al..."
96,“You have to write the book that wants to be w...,Madeleine L'Engle,"books,children,difficult,grown-ups,write,write...","November 29, 1918","in New York City, New York, The United States",Madeleine L'Engle was an American writer best ...
97,“Never tell the truth to people who are not wo...,Mark Twain,truth,"November 30, 1835","in Florida, Missouri, The United States","Samuel Langhorne Clemens, better known by his ..."
98,"“A person's a person, no matter how small.”",Dr. Seuss,inspirational,"March 02, 1904","in Springfield, MA, The United States",Theodor Seuss Geisel was born 2 March 1904 in ...


In [116]:
final_df.to_csv('data.csv', index=False)


In [117]:
len(final_df)

100